# CryptoCompare example

Crypto Currency Arbitrage Trading Back Testing within CryptoCompare.

This document discribed how to get data from CryptoCompare to analyze arbitrage opportunity.

# 1. Calc the common coins in Okex, Binance, Huobi and gate

In [1]:
# get the exchange and coins data from CryptoCompare

import requests

exchangeCoinFile = open("../data/cryptocompare/cryptocompare.exchange.json","w+")

exchangeCoinQuerryURL = "https://min-api.cryptocompare.com/data/all/exchanges"

exchangeCoinResponse = requests.request("GET", exchangeCoinQuerryURL)

exchangeCoinFile.write(exchangeCoinResponse.text)

exchangeCoinFile.close

print("done")

done


In [2]:
# calc the common coins in Okex Binance Huobi Gate

import json

exchangeCoinFile = "../data/cryptocompare/cryptocompare.exchange.json"

# Reading data back
with open(exchangeCoinFile, 'r') as f:
    exchangeCoinData = json.load(f)

okexCoin = exchangeCoinData["OKEX"]
binanceCoin = exchangeCoinData["Binance"]
huobiCoin = exchangeCoinData["HuobiPro"]
gateCoin = exchangeCoinData["Gateio"]

okexBinanceCommonKeys = okexCoin.keys() & binanceCoin.keys()
okexHuobiCommonKeys = okexCoin.keys() & huobiCoin.keys()
okexGateCommonKeys = okexCoin.keys() & gateCoin.keys()
binanceHuobicommonKeys = binanceCoin.keys() & huobiCoin.keys()
binanceGateCommonKeys = binanceCoin.keys() & gateCoin.keys()
huobiGateCommonKeys = huobiCoin.keys() & gateCoin.keys()

okexBinanceCoin = {}
okexHuobiCoin = {}
okexGateCoin = {}
binanceHuobiCoin = {}
binanceGateCoin = {}
huobiGateCoin = {}

for key in okexBinanceCommonKeys:
    okexBinanceCoin[key] = list(set(okexCoin[key]).intersection(set(binanceCoin[key])))
for key in okexHuobiCommonKeys:
    okexHuobiCoin[key] = list(set(okexCoin[key]).intersection(set(huobiCoin[key])))
for key in okexGateCommonKeys:
    okexGateCoin[key] = list(set(okexCoin[key]).intersection(set(gateCoin[key])))
for key in binanceHuobicommonKeys:
    binanceHuobiCoin[key] = list(set(binanceCoin[key]).intersection(set(huobiCoin[key])))
for key in binanceGateCommonKeys:
    binanceGateCoin[key] = list(set(binanceCoin[key]).intersection(set(gateCoin[key])))
for key in huobiGateCommonKeys:
    huobiGateCoin[key] = list(set(huobiCoin[key]).intersection(set(gateCoin[key])))

okexBinanceCoinFile = "../data/cryptocompare/cryptocompare.exchange.okexBinanceCoin.json"
okexHuobiCoinFile = "../data/cryptocompare/cryptocompare.exchange.okexHuobiCoin.json"
okexGateCoinFile = "../data/cryptocompare/cryptocompare.exchange.okexGateCoin.json"
binanceHuobiCoinFile = "../data/cryptocompare/cryptocompare.exchange.binanceHuobiCoin.json"
binanceGateCoinFile = "../data/cryptocompare/cryptocompare.exchange.binanceGateCoin.json"
huobiGateCoinFile = "../data/cryptocompare/cryptocompare.exchange.huobiGateCoin.json"

with open(okexBinanceCoinFile, 'w+') as f:
    json.dump(okexBinanceCoin, f)
with open(okexHuobiCoinFile, 'w+') as f:
    json.dump(okexHuobiCoin, f)
with open(okexGateCoinFile, 'w+') as f:
    json.dump(okexGateCoin, f)
with open(binanceHuobiCoinFile, 'w+') as f:
    json.dump(binanceHuobiCoin, f)
with open(binanceGateCoinFile, 'w+') as f:
    json.dump(binanceGateCoin, f)
with open(huobiGateCoinFile, 'w+') as f:
    json.dump(huobiGateCoin, f)

print("done")

done


In [3]:
# define common function
import sys
import json
import time
import requests
import numpy as np
import pandas as pd

def calcDayArbitrage(fSymbol, tSymbol, xExchange, xFee, yExchange, yFee, timeStamp):
    # handle timestamp (senconds to hour)
    timeStamp = timeStamp - timeStamp%1440
    # cryptoCompare api
    url = "https://min-api.cryptocompare.com/data/histominute"
    # querry str
    xQuerryStr =  {"fsym":fSymbol,"tsym":tSymbol,"limit":"1440","aggregate":"1",  
               "e":xExchange,"toTs":timeStamp}
    yQuerryStr =  {"fsym":fSymbol,"tsym":tSymbol,"limit":"1440","aggregate":"1",  
               "e":yExchange,"toTs":timeStamp}
    # request response
    try:
        xResponse = requests.request("GET", url, params=xQuerryStr)
        yResponse = requests.request("GET", url, params=yQuerryStr)
        if xResponse.status_code == requests.codes.ok and yResponse.status_code == requests.codes.ok:
            # get xDataFrame, yDataFrame
            xDataFrame = pd.DataFrame(xResponse.json()["Data"])
            yDataFrame = pd.DataFrame(yResponse.json()["Data"])
            # calc measurements
            measurements = calcMeasurements(xDataFrame, yDataFrame, xFee, yFee)
            chance = 0
            profit = 0
            volume = 0
            candy = 0
            for index in range(len(measurements)):
                if measurements.loc[index,"profit"] > 0:
                    if measurements.loc[index,"difference"] != 0 and candy == 0:
                        if measurements.loc[index,"difference"] > 0:
                            candy = 1
                        else:
                            candy = -1
                        chance = 1
                        profit = measurements.loc[index,"profit"]
                        volume = measurements.loc[index,"volume"]
                    if measurements.loc[index,"difference"] > 0 and candy < 0:
                        candy = 1
                        chance = chance + 1
                        profit = profit + measurements.loc[index,"profit"]
                        volume = volume + measurements.loc[index,"volume"]
                    if measurements.loc[index,"difference"] < 0 and candy > 0:
                        candy = -1
                        chance = chance + 1
                        profit = profit + measurements.loc[index,"profit"]
                        volume = volume + measurements.loc[index,"volume"]
            return [chance, profit, volume]
        else:
            return [-1, -1, -1]
    except requests.exceptions.RequestException as e:
        print(e)
        sys.exit(1)

def calcHourArbitrage(fSymbol, tSymbol, xExchange, xFee, yExchange, yFee, timeStamp):
    # handle timestamp (senconds to hour)
    timeStamp = timeStamp - timeStamp%60
    # cryptoCompare api
    url = "https://min-api.cryptocompare.com/data/histominute"
    # querry str
    xQuerryStr =  {"fsym":fSymbol,"tsym":tSymbol,"limit":"60","aggregate":"1",  
               "e":xExchange,"toTs":timeStamp}
    yQuerryStr =  {"fsym":fSymbol,"tsym":tSymbol,"limit":"60","aggregate":"1",  
               "e":yExchange,"toTs":timeStamp}
    # request response
    try:
        xResponse = requests.request("GET", url, params=xQuerryStr)
        yResponse = requests.request("GET", url, params=yQuerryStr)
        if xResponse.status_code == requests.codes.ok and yResponse.status_code == requests.codes.ok:
            # get xDataFrame, yDataFrame
            xDataFrame = pd.DataFrame(xResponse.json()["Data"])
            yDataFrame = pd.DataFrame(yResponse.json()["Data"])
            # calc measurements
            measurements = calcMeasurements(xDataFrame, yDataFrame, xFee, yFee)
            chance = 0
            profit = 0
            volume = 0
            candy = 0
            for index in range(len(measurements)):
                if measurements.loc[index,"profit"] > 0:
                    if measurements.loc[index,"difference"] != 0 and candy == 0:
                        if measurements.loc[index,"difference"] > 0:
                            candy = 1
                        else:
                            candy = -1
                        chance = 1
                        profit = measurements.loc[index,"profit"]
                        volume = measurements.loc[index,"volume"]
                    if measurements.loc[index,"difference"] > 0 and candy < 0:
                        candy = 1
                        chance = chance + 1
                        profit = profit + measurements.loc[index,"profit"]
                        volume = volume + measurements.loc[index,"volume"]
                    if measurements.loc[index,"difference"] < 0 and candy > 0:
                        candy = -1
                        chance = chance + 1
                        profit = profit + measurements.loc[index,"profit"]
                        volume = volume + measurements.loc[index,"volume"]
            return [chance, profit, volume]
        else:
            return [-1, -1, -1]
    except requests.exceptions.RequestException as e:
        print(e)
        sys.exit(1)

def calcMeasurements(xDataFrame, yDataFrame, xFee, yFee):
    # checkout dataFrame
    xDataFrame.set_index(["time"], inplace=True)
    yDataFrame.set_index(["time"], inplace=True)
    # get the intersection
    ixs = xDataFrame.index.intersection(yDataFrame.index)
    # get x and y sample
    xDataSample = xDataFrame.loc[ixs]
    yDataSample = yDataFrame.loc[ixs]
    xDataSample.reset_index(drop=False, inplace=True)
    yDataSample.reset_index(drop=False, inplace=True)
    # calc measurements
    measurements = pd.DataFrame(columns=("time", "difference", "profit", "volume"))
    measurements["time"] = pd.Series(ixs)
    measurements["difference"] = pd.Series(xDataSample["close"] - yDataSample["close"])
    measurements["profit"] = pd.Series(np.abs(xDataSample["close"] - yDataSample["close"]) 
                                - (xDataSample["close"] * xFee + yDataSample["close"] * yFee))
    measurements["volume"] = pd.Series(np.minimum(xDataSample["volumeto"], yDataSample["volumeto"]))
    return measurements

print("done")

done


In [4]:
 # calc arbitrage opportunity in Okex and Binance

timeStr = "2018-11-4 18:00:00"
okexBinanceCoinFile = "../data/cryptocompare/cryptocompare.exchange.okexBinanceCoin.json"

# Reading data back
with open(okexBinanceCoinFile, 'r') as f:
    okexBinanceCoin = json.load(f)

# Calc timeStamp
timeStamp = int(time.mktime(time.strptime(timeStr, "%Y-%m-%d %H:%M:%S")))

# chance, profit, volume = calcHourArbitrage("DNT", "ETH", "OKEX", 0.002, "Binance", 0.001, timeStamp)

for fsym in okexBinanceCoin:
    for tsym in okexBinanceCoin[fsym]:
        [chance, profit, volume] = calcHourArbitrage(fsym, tsym, "OKEX", 0.002, "Binance", 0.001, timeStamp)        
        print(fsym,'vs',tsym)
        print("   chance =", chance)
        print("   profit =", profit)
        print("   volume =", volume)

INS vs BTC
   chance = 8
   profit = 1.8084600000000198e-06
   volume = 0.16395699999999996
INS vs ETH
   chance = 12
   profit = 0.0002079060000000007
   volume = 0.42716361
POE vs BTC
   chance = 1
   profit = 3.799999999999903e-09
   volume = 0.0
POE vs ETH
   chance = 1
   profit = 4.426099999999938e-07
   volume = 0.0
DGD vs BTC
   chance = 2
   profit = 4.4306999999999744e-05
   volume = 0.0
DGD vs ETH
   chance = 2
   profit = 0.0007722000000000019
   volume = 0.0
EVX vs BTC
   chance = 1
   profit = 6.45e-07
   volume = 0.0
EVX vs ETH
   chance = 1
   profit = 4.5091000000000094e-05
   volume = 0.0
GTO vs BTC
   chance = 1
   profit = 4.880999999999923e-08
   volume = 1.05
GTO vs ETH
   chance = 1
   profit = 1.6115999999999546e-06
   volume = 11.93
OST vs BTC
   chance = 3
   profit = 3.904000000000233e-08
   volume = 0.0
OST vs ETH
   chance = 2
   profit = 5.7147999999999855e-06
   volume = 0.4514
BTG vs BTC
   chance = 4
   profit = 2.881399999999974e-05
   volume = 0.03199

YOYOW vs BTC
   chance = 1
   profit = 6.211000000000008e-08
   volume = 0.0
YOYOW vs ETH
   chance = 1
   profit = 2.8395000000000014e-06
   volume = 0.0
PPT vs BTC
   chance = 1
   profit = 6.564600000000039e-06
   volume = 0.0
PPT vs ETH
   chance = 1
   profit = 0.00017478000000000103
   volume = 0.0
OAX vs BTC
   chance = 1
   profit = 3.4923700000000028e-06
   volume = 0.0
OAX vs ETH
   chance = 1
   profit = 0.0007680887999999999
   volume = 0.0
MTH vs BTC
   chance = 2
   profit = 5.799999999996187e-10
   volume = 0.0
MTH vs ETH
   chance = 1
   profit = 5.461999999999986e-06
   volume = 0.0
QTUM vs BTC
   chance = 1
   profit = 4.5640000000000147e-07
   volume = 0.0001839
QTUM vs USDT
   chance = 0
   profit = 0
   volume = 0
QTUM vs ETH
   chance = 10
   profit = 0.00034340000000000027
   volume = 2.606285
NULS vs BTC
   chance = 1
   profit = 1.8768000000000042e-06
   volume = 0.0
NULS vs USDT
   chance = 1
   profit = 0.006629999999999787
   volume = 0.0
NULS vs ETH
   chan

In [5]:
 # calc arbitrage opportunity in Okex and Huobi

timeStr = "2018-11-4 18:00:00"
okexHuobiCoinFile = "../data/cryptocompare/cryptocompare.exchange.okexHuobiCoin.json"

# Reading data back
with open(okexHuobiCoinFile, 'r') as f:
    okexHuobiCoin = json.load(f)

# Calc timeStamp
timeStamp = int(time.mktime(time.strptime(timeStr, "%Y-%m-%d %H:%M:%S")))

# chance, profit, volume = calcHourArbitrage("DNT", "ETH", "OKEX", 0.002, "Huobi", 0.001, timeStamp)

for fsym in okexHuobiCoin:
    for tsym in okexHuobiCoin[fsym]:
        [chance, profit, volume] = calcHourArbitrage(fsym, tsym, "OKEX", 0.002, "HuobiPro", 0.001, timeStamp)        
        print(fsym,'vs',tsym)
        print("   chance =", chance)
        print("   profit =", profit)
        print("   volume =", volume)

EVX vs BTC
   chance = 1
   profit = 3.1532999999999466e-07
   volume = 0.0
EVX vs ETH
   chance = 1
   profit = 1.9065000000000113e-05
   volume = 0.0
DGD vs BTC
   chance = 2
   profit = 1.2336999999999828e-05
   volume = 0.0
DGD vs ETH
   chance = 2
   profit = 0.00017180000000001242
   volume = 0.0
BCD vs BTC
   chance = 1
   profit = 0.00028662659999999994
   volume = 0.0
OST vs BTC
   chance = 3
   profit = 3.900000000000232e-08
   volume = 0.0
OST vs ETH
   chance = 2
   profit = 8.11199999999999e-06
   volume = 0.0003844
BTG vs BTC
   chance = 6
   profit = 6.964599999999835e-05
   volume = 0.08134999999999999
SOC vs BTC
   chance = 6
   profit = 2.7510000000000694e-08
   volume = 4.6469999999999994e-05
SOC vs USDT
   chance = 1
   profit = 6.55999999999994e-05
   volume = 0.0
SOC vs ETH
   chance = 1
   profit = 1.8409999999996497e-08
   volume = 8.218e-05
SWFTC vs BTC
   chance = 0
   profit = 0
   volume = 0
SWFTC vs ETH
   chance = 6
   profit = 1.8581999999999797e-07
   vo

QTUM vs USDT
   chance = 0
   profit = 0
   volume = 0
QTUM vs ETH
   chance = 6
   profit = 0.00023800999999999673
   volume = 0.037792
ABT vs BTC
   chance = 3
   profit = 3.187999999999893e-08
   volume = 0.00783
ABT vs ETH
   chance = 2
   profit = 6.457000000000945e-07
   volume = 0.01107
ETC vs BTC
   chance = 0
   profit = 0
   volume = 0
ETC vs USDT
   chance = 0
   profit = 0
   volume = 0
ITC vs BTC
   chance = 1
   profit = 1.9199999999956523e-09
   volume = 0.004376
ITC vs USDT
   chance = 1
   profit = 6.930000000000618e-05
   volume = 106.14
ITC vs ETH
   chance = 2
   profit = 2.763999999999893e-06
   volume = 0.8283
BCX* vs BTC
   chance = 1
   profit = 5.0913e-07
   volume = 0.0
XMR vs BTC
   chance = 1
   profit = 0.000510989999999998
   volume = 0.0
XMR vs ETH
   chance = 1
   profit = 0.01804799999999995
   volume = 0.006538
MANA vs BTC
   chance = 1
   profit = 3.769999999999616e-09
   volume = 0.00229
MANA vs ETH
   chance = 3
   profit = 9.647000000000526e-07
   

In [6]:
 # calc arbitrage opportunity in Okex and Gate

timeStr = "2018-11-4 18:00:00"
okexGateCoinFile = "../data/cryptocompare/cryptocompare.exchange.okexGateCoin.json"

# Reading data back
with open(okexGateCoinFile, 'r') as f:
    okexGateCoin = json.load(f)

# Calc timeStamp
timeStamp = int(time.mktime(time.strptime(timeStr, "%Y-%m-%d %H:%M:%S")))

# chance, profit, volume = calcHourArbitrage("DNT", "ETH", "OKEX", 0.002, "Gate", 0.001, timeStamp)

for fsym in okexGateCoin:
    for tsym in okexGateCoin[fsym]:
        [chance, profit, volume] = calcHourArbitrage(fsym, tsym, "OKEX", 0.002, "Gateio", 0.001, timeStamp)        
        print(fsym,'vs',tsym)
        print("   chance =", chance)
        print("   profit =", profit)
        print("   volume =", volume)

DGD vs USDT
   chance = 1
   profit = 1.1786999999999972
   volume = 0
DGD vs ETH
   chance = 1
   profit = 0.0018884000000000023
   volume = 0
BCD vs BTC
   chance = 8
   profit = 1.4360200000000034e-05
   volume = 0.25293699999999997
BCD vs USDT
   chance = 11
   profit = 0.2079799999999994
   volume = 40054.020000000004
TCT vs USDT
   chance = 9
   profit = 5.690000000000499e-05
   volume = 5029.634
TCT vs ETH
   chance = 3
   profit = 6.907999999999968e-07
   volume = 0.0
MITH vs USDT
   chance = 1
   profit = 2.3199999999967387e-05
   volume = 0.0
MITH vs ETH
   chance = 1
   profit = 3.1670000000000613e-06
   volume = 0.0
BTG vs BTC
   chance = 1
   profit = 0.00012331199999999996
   volume = 0.0
BTG vs USDT
   chance = 1
   profit = 0.12306000000000085
   volume = 0.0
GTC vs BTC
   chance = 1
   profit = 6.99642e-06
   volume = 0.0
GTC vs USDT
   chance = 1
   profit = 0.052033800000000005
   volume = 0.0
GTC vs ETH
   chance = 1
   profit = 7.924360000000001e-05
   volume = 0.0

PPT vs USDT
   chance = 1
   profit = 2.2866
   volume = 0.0
PPT vs ETH
   chance = 1
   profit = 0.00760734
   volume = 0.0
BNT vs USDT
   chance = 1
   profit = 0.8551200000000001
   volume = 0
BNT vs ETH
   chance = 1
   profit = 0.000176387
   volume = 0.0
RFR vs USDT
   chance = 1
   profit = 0.00011217300000000038
   volume = 0.0
RFR vs ETH
   chance = 1
   profit = 9.329999999999988e-07
   volume = 0.0
QTUM vs BTC
   chance = 1
   profit = 3.5060000000005323e-07
   volume = 0.0
QTUM vs USDT
   chance = 0
   profit = 0
   volume = 0
QTUM vs ETH
   chance = 1
   profit = 0.00011146000000000001
   volume = 0.0
ABT vs USDT
   chance = 1
   profit = 0.0019458000000000023
   volume = 0.0
ABT vs ETH
   chance = 1
   profit = 5.736000000000262e-07
   volume = 0.0
BCX* vs BTC
   chance = 1
   profit = 3.8924999999999997e-07
   volume = 0
ETC vs BTC
   chance = 2
   profit = 7.1980000000001715e-06
   volume = 0.0
ETC vs USDT
   chance = 1
   profit = 0.011700000000000925
   volume = 1813.

In [7]:
 # calc arbitrage opportunity in binance and Huobi

timeStr = "2018-11-4 18:00:00"
binanceHuobiCoinFile = "../data/cryptocompare/cryptocompare.exchange.binanceHuobiCoin.json"

# Reading data back
with open(binanceHuobiCoinFile, 'r') as f:
    binanceHuobiCoin = json.load(f)

# Calc timeStamp
timeStamp = int(time.mktime(time.strptime(timeStr, "%Y-%m-%d %H:%M:%S")))

# chance, profit, volume = calcHourArbitrage("DNT", "ETH", "binance", 0.002, "Huobi", 0.001, timeStamp)

for fsym in binanceHuobiCoin:
    for tsym in binanceHuobiCoin[fsym]:
        [chance, profit, volume] = calcHourArbitrage(fsym, tsym, "Binance", 0.002, "HuobiPro", 0.001, timeStamp)        
        print(fsym,'vs',tsym)
        print("   chance =", chance)
        print("   profit =", profit)
        print("   volume =", volume)

EVX vs BTC
   chance = 7
   profit = 1.3717800000000127e-06
   volume = 0.00130748
EVX vs ETH
   chance = 1
   profit = 1.817099999999998e-05
   volume = 0.001023
DGD vs BTC
   chance = 2
   profit = 3.232400000000075e-05
   volume = 0.0
DGD vs ETH
   chance = 4
   profit = 0.0006545999999999916
   volume = 0.007501
BCD vs BTC
   chance = 1
   profit = 0.000276807
   volume = 0.0
OST vs BTC
   chance = 4
   profit = 4.5840000000002044e-08
   volume = 0.00278666
OST vs ETH
   chance = 6
   profit = 2.7082999999999576e-06
   volume = 0.0010513
BTG vs BTC
   chance = 7
   profit = 7.211999999999961e-05
   volume = 0.24697899999999998
GXS vs BTC
   chance = 3
   profit = 4.899999999999771e-07
   volume = 0.04082848
GXS vs ETH
   chance = 2
   profit = 2.463099999999995e-05
   volume = 0.0
BCH vs BTC
   chance = 0
   profit = 0
   volume = 0
BCH vs USDT
   chance = 0
   profit = 0
   volume = 0
TNT vs BTC
   chance = 3
   profit = 2.1350000000000175e-08
   volume = 0.00059505
TNT vs ETH
   

MANA vs ETH
   chance = 4
   profit = 1.8094999999999386e-06
   volume = 9.857394
THETA vs BTC
   chance = 2
   profit = 2.4820000000000642e-08
   volume = 2.111e-05
THETA vs ETH
   chance = 3
   profit = 1.7413000000000324e-06
   volume = 0.024096
ADA vs BTC
   chance = 0
   profit = 0
   volume = 0
ADA vs USDT
   chance = 0
   profit = 0
   volume = 0
ADA vs ETH
   chance = 2
   profit = 1.0065999999999695e-06
   volume = 1.38
RCN vs BTC
   chance = 3
   profit = 1.505000000000112e-08
   volume = 0.00225298
RCN vs ETH
   chance = 3
   profit = 9.767000000000041e-07
   volume = 0.0
ZRX vs BTC
   chance = 0
   profit = 0
   volume = 0
ZRX vs ETH
   chance = 1
   profit = 4.097999999999521e-06
   volume = 0.002665
POWR vs BTC
   chance = 2
   profit = 1.9060000000000053e-08
   volume = 0.012399
POWR vs ETH
   chance = 1
   profit = 1.3049999999999274e-06
   volume = 0.003985
ENG vs BTC
   chance = 2
   profit = 1.5525000000000986e-07
   volume = 4.494e-05
ENG vs ETH
   chance = 6
   pro

In [8]:
 # calc arbitrage opportunity in binance and Gate

timeStr = "2018-11-4 18:00:00"
binanceGateCoinFile = "../data/cryptocompare/cryptocompare.exchange.binanceGateCoin.json"

# Reading data back
with open(binanceGateCoinFile, 'r') as f:
    binanceGateCoin = json.load(f)

# Calc timeStamp
timeStamp = int(time.mktime(time.strptime(timeStr, "%Y-%m-%d %H:%M:%S")))

# chance, profit, volume = calcHourArbitrage("DNT", "ETH", "binance", 0.002, "Gate", 0.001, timeStamp)

for fsym in binanceGateCoin:
    for tsym in binanceGateCoin[fsym]:
        [chance, profit, volume] = calcHourArbitrage(fsym, tsym, "Binance", 0.002, "Gateio", 0.001, timeStamp)        
        print(fsym,'vs',tsym)
        print("   chance =", chance)
        print("   profit =", profit)
        print("   volume =", volume)

DGD vs ETH
   chance = 1
   profit = 0.0026867999999999974
   volume = 0.0
REP vs ETH
   chance = 2
   profit = 0.0025106699999999944
   volume = 0.0
BCD vs BTC
   chance = 4
   profit = 1.6467999999999995e-05
   volume = 0.004832
OST vs ETH
   chance = 2
   profit = 3.0599999999998953e-07
   volume = 0.0
BTG vs BTC
   chance = 1
   profit = 0.00013433400000000057
   volume = 0.0
GXS vs BTC
   chance = 1
   profit = 3.360299999999989e-06
   volume = 0.0
FUN vs ETH
   chance = 2
   profit = 2.1273999999999906e-07
   volume = 0.0
MDA vs ETH
   chance = 1
   profit = 0.00024197600000000008
   volume = 0.0
WINGS vs ETH
   chance = 1
   profit = 2.824800000000023e-06
   volume = 0.08732
BCH vs BTC
   chance = 4
   profit = 0.0003890999999999984
   volume = 0.0001408
BCH vs USDT
   chance = 5
   profit = 1.4075199999999992
   volume = 11387.41
KNC vs ETH
   chance = 2
   profit = 3.0777999999999637e-05
   volume = 0.0
LRC vs BTC
   chance = 2
   profit = 2.2020000000000445e-08
   volume = 0.

LTC vs BTC
   chance = 4
   profit = 4.8093999999998825e-05
   volume = 0.053973999999999994
LTC vs USDT
   chance = 3
   profit = 0.028850000000005094
   volume = 1092.1


In [9]:
 # calc arbitrage opportunity in huobi and Gate

timeStr = "2018-11-4 18:00:00"
huobiGateCoinFile = "../data/cryptocompare/cryptocompare.exchange.huobiGateCoin.json"

# Reading data back
with open(huobiGateCoinFile, 'r') as f:
    huobiGateCoin = json.load(f)

# Calc timeStamp
timeStamp = int(time.mktime(time.strptime(timeStr, "%Y-%m-%d %H:%M:%S")))

# chance, profit, volume = calcHourArbitrage("DNT", "ETH", "huobi", 0.002, "Gate", 0.001, timeStamp)

for fsym in huobiGateCoin:
    for tsym in huobiGateCoin[fsym]:
        [chance, profit, volume] = calcHourArbitrage(fsym, tsym, "HuobiPro", 0.002, "Gateio", 0.001, timeStamp)        
        print(fsym,'vs',tsym)
        print("   chance =", chance)
        print("   profit =", profit)
        print("   volume =", volume)

DGD vs ETH
   chance = 1
   profit = 0.0020880000000000083
   volume = 0.0
BCD vs BTC
   chance = 1
   profit = 0.00029054299999999993
   volume = 0.0
OST vs ETH
   chance = 2
   profit = 1.4054000000000162e-06
   volume = 0.0
BTG vs BTC
   chance = 1
   profit = 0.00012030600000000044
   volume = 0.0
GXS vs BTC
   chance = 1
   profit = 3.159899999999984e-06
   volume = 0.0
MDS vs USDT
   chance = 1
   profit = 0.00011243000000000041
   volume = 0.0
MDS vs ETH
   chance = 1
   profit = 7.620599999999972e-07
   volume = 0.0
MTN* vs ETH
   chance = 1
   profit = 1.0667999999999933e-06
   volume = 0.0
BCH vs BTC
   chance = 3
   profit = 0.0004514799999999888
   volume = 0.0001408
BCH vs USDT
   chance = 3
   profit = 0.9350900000000861
   volume = 5918.0
KNC vs ETH
   chance = 2
   profit = 2.3736000000000008e-05
   volume = 0.0007353
TNT vs ETH
   chance = 4
   profit = 1.0478000000000211e-06
   volume = 0.0
LINK vs ETH
   chance = 1
   profit = 1.9341000000000112e-05
   volume = 0.0
S

POWR vs ETH
   chance = 1
   profit = 3.0060000000001115e-07
   volume = 0.0
XMR vs BTC
   chance = 0
   profit = 0
   volume = 0
NAS vs BTC
   chance = 1
   profit = 1.0153000000000142e-06
   volume = 0.0
NAS vs USDT
   chance = 2
   profit = 0.011210000000000019
   volume = 276.06
NAS vs ETH
   chance = 1
   profit = 1.1740000000004499e-06
   volume = 0.0
SALT vs ETH
   chance = 1
   profit = 0.00029108699999999977
   volume = 0.0
REQ vs ETH
   chance = 1
   profit = 1.422000000000218e-07
   volume = 0.0
LTC vs BTC
   chance = 6
   profit = 5.915799999999564e-05
   volume = 0.13579599999999997
LTC vs USDT
   chance = 1
   profit = 0.01946999999999971
   volume = 587.29
